In [1]:
# import packages
from dsets import CounterFactDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Dict, List, Tuple
import importlib
import numpy as np
import torch
from transformers import set_seed
import time
import os

from rome import ROMEHyperParams
from utils.tools import (
    ModelAndTokenizer,
    find_token_range,
)
import tools
importlib.reload(tools)
from tools import (
    execute_rome,
    test_case,
    run_rome,
    model_reset,
    add_knockout_attn,
    attn_reset,
    noise_test,
    plot_casual_trace,
    plot_attn_knockout
)

In [2]:
# Load model
set_seed(42)
torch.cuda.set_device(7)
MODEL_PATH="/public/home/ljt/hf_models/gpt-j-6b"
# MODEL_PATH="EleutherAI/gpt-j-6b"
PARAMA_PATH="hparams/ROME/EleutherAI_gpt-j-6B.json"

model_name = "gpt2-j" if MODEL_PATH.endswith('6b') else "gpt2-xl"
edit_flag, knockout = False, False
hparams = ROMEHyperParams.from_json(PARAMA_PATH)
tok = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).cuda()
tok.pad_token = tok.eos_token
mt = ModelAndTokenizer(
    MODEL_PATH,
    model,
    low_cpu_mem_usage=False,
    torch_dtype=(torch.float16 if "20b" in model_name else None),
)
noise_dict = {
    "gpt2-j": 0.09413417056202888,
    "gpt2-xl": 0.13462981581687927
}
noise_level = noise_dict[model_name]

/public/home/ljt/anaconda3/envs/rome/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# some cases for editing and testing
case = dict(prompt = 'Eiffel Tower is located in',
            subject = 'Eiffel Tower',
            target_true = ' Paris',
            target_new = ' New York',
            test_prompt = 'Eiffel Tower is famous. Pyramids is in',
            test_true = ' Egypt')

# before editing test
print("edit flag", edit_flag)
test_case(case, tok, model)

edit flag False
Testing with original prompt:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated output: Eiffel Tower is located in the heart of Paris, France. It is the
Top 3 words and probabilities: [('Ġthe', 0.436737060546875), ('ĠParis', 0.33399298787117004), ('ĠFrance', 0.04520353302359581)]
Probability and output for specified word  Paris (target_true): 0.33399298787117004
Probability for specified word  New York (target_new): 0.0004106156702619046
----------------------------------------------------------------------------------------------------
Testing with test prompt:
Generated output: Eiffel Tower is famous. Pyramids is in Egypt. The Great Wall of China is in China
Top 3 words and probabilities: [('ĠEgypt', 0.5749076008796692), ('Ġthe', 0.08010730147361755), ('ĠAfrica', 0.03189391270279884)]
Probability and output for specified word  Egypt (test_true): 0.5749076008796692
Probability and output for specified word  New York (target_new): 0.0001971092278836295


In [4]:
# Do edit
# assert edit_flag == False
assert knockout == False
weights_copy = run_rome(case, hparams, model, tok)
edit_flag = True

Executing ROME algorithm for the update: [Eiffel Tower is located in] -> [ New York]
Cached context templates ['{}', 'Q: . {}', 'Q: . {}', 'The present invention relates. {}', 'The role of the. {}', '\n \n-. {}', 'A new report from. {}', 'Q: . {}', 'Q: . {}', '\n \n=. {}', 'Q: . {}', 'The present invention relates to a method for producing. {}', ' Ask HN: Is there any. {}', " Show HN: I'm looking. {}", 'Q: Is there a way to. {}', ' Show HN: The best way. {}', 'Q: How to make a list. {}', 'Q: How to use a function. {}', 'Q: How do you get the. {}', 'Q: Can I get the current. {}', 'Q: What is a good way. {}']
Computing left vector (u)...
Selected u projection object Eiffel Tower


Using the latest cached version of the module from /public/home/ljt/.cache/huggingface/modules/datasets_modules/datasets/wikipedia/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559 (last modified on Sun Mar 10 18:45:50 2024) since it couldn't be found locally at wikipedia., or remotely on the Hugging Face Hub.


Retrieving inverse covariance statistics for _public_home_ljt_hf_models_gpt-j-6b @ transformer.h.5.mlp.fc_out. The result will be cached to avoid repetitive computation.
../../../../data/rome/status/EleutherAI_gpt-j-6B/wikipedia_stats/transformer.h.5.mlp.fc_out_float32_mom2_100000.npz
Attempting to download EleutherAI_gpt-j-6B/wikipedia_stats/transformer.h.5.mlp.fc_out_float32_mom2_100000.npz from https://rome.baulab.info/data/stats/EleutherAI_gpt-j-6B/wikipedia_stats/transformer.h.5.mlp.fc_out_float32_mom2_100000.npz.
Unable to download due to <urlopen error [Errno -2] Name or service not known>. Computing locally....


ValueError: BuilderConfig 20200501.en not found. Available: ['20220301.aa', '20220301.ab', '20220301.ace', '20220301.ady', '20220301.af', '20220301.ak', '20220301.als', '20220301.am', '20220301.an', '20220301.ang', '20220301.ar', '20220301.arc', '20220301.arz', '20220301.as', '20220301.ast', '20220301.atj', '20220301.av', '20220301.ay', '20220301.az', '20220301.azb', '20220301.ba', '20220301.bar', '20220301.bat-smg', '20220301.bcl', '20220301.be', '20220301.be-x-old', '20220301.bg', '20220301.bh', '20220301.bi', '20220301.bjn', '20220301.bm', '20220301.bn', '20220301.bo', '20220301.bpy', '20220301.br', '20220301.bs', '20220301.bug', '20220301.bxr', '20220301.ca', '20220301.cbk-zam', '20220301.cdo', '20220301.ce', '20220301.ceb', '20220301.ch', '20220301.cho', '20220301.chr', '20220301.chy', '20220301.ckb', '20220301.co', '20220301.cr', '20220301.crh', '20220301.cs', '20220301.csb', '20220301.cu', '20220301.cv', '20220301.cy', '20220301.da', '20220301.de', '20220301.din', '20220301.diq'

In [ ]:
# after editing test
print("edit flag", edit_flag)
test_case(case, tok, model)